In [ ]:
!pip install pandas sqlalchemy pymysql openpyxl

In [3]:
import pandas as pd
from sqlalchemy import create_engine, text

# 1. 엑셀 파일 로드
file_path = r'C:\Users\k2003\OneDrive\문서\업무\의뢰서목록.xlsx'
df = pd.read_excel(file_path)

# 2. 데이터 전처리
# 배정일자: "YYYY-MM-DD" 텍스트 형식으로 변환
if '배정일자' in df.columns:
    df['배정일자'] = pd.to_datetime(df['배정일자']).dt.strftime('%Y-%m-%d')
else:
    df['배정일자'] = None

# 업데이트에 필요한 컬럼들만 추출 (의뢰번호 기준)
# 엑셀 파일에 '담당자', '미인정' 컬럼이 반드시 있어야 합니다.
update_columns = ['의뢰번호', '배정일자', '담당자', '미인정']

# 엑셀에 해당 컬럼이 있는지 확인 후 추출 (의뢰번호가 없는 행은 제거)
df_update = df[update_columns].dropna(subset=['의뢰번호'])

# 3. DB 연결 설정
user = "kcqt_kyj"
password = "1977519"
host = "221.155.228.179"
port = "3306"
db_name = "kcqt_qyalit"

engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{db_name}')

# 4. 업데이트 실행
try:
    with engine.begin() as conn:
        # A. 임시 테이블 생성 (필요한 컬럼들 포함)
        # d_val(배정일자), manager(담당자), unapproved(미인정)
        conn.execute(text("""
            CREATE TEMPORARY TABLE temp_update (
                u_id VARCHAR(50), 
                d_val VARCHAR(20),
                manager VARCHAR(50),
                unapproved VARCHAR(100)
            )
        """))
        
        # B. 엑셀 데이터를 임시 테이블에 삽입
        data_list = df_update.values.tolist()
        if not data_list:
            print("⚠️ 업데이트할 데이터가 엑셀에 없습니다.")
        else:
            conn.execute(
                text("""
                    INSERT INTO temp_update (u_id, d_val, manager, unapproved) 
                    VALUES (:u_id, :d_val, :manager, :unapproved)
                """),
                [{"u_id": str(r[0]), "d_val": str(r[1]), "manager": str(r[2]), "unapproved": str(r[3])} for r in data_list]
            )
            
            # C. JOIN 업데이트 실행 (3개 컬럼 동시 업데이트)
            # 원본 테이블(csi_receipts)의 컬럼명과 매칭시킵니다.
            sql = """
                UPDATE csi_receipts AS target
                INNER JOIN temp_update AS source ON target.의뢰번호 = source.u_id
                SET target.배정일자 = source.d_val,
                    target.담당자 = source.manager,
                    target.미인정 = source.unapproved
            """
            result = conn.execute(text(sql))
            print(f"✅ 업데이트 성공: 총 {result.rowcount}개의 행(배정일자, 담당자, 미인정)이 수정되었습니다.")

except Exception as e:
    print(f"❌ 오류 발생: {e}")

✅ 업데이트 성공: 총 1306개의 행(배정일자, 담당자, 미인정)이 수정되었습니다.
